## Course Project -- Milestone 2
### Elizabeth Earl 
### DSC550-T301

In [177]:
import pandas as pd
books = pd.read_csv('books09_19.csv')
electronics = pd.read_csv('electronics2021.csv')

In [195]:
books.head()

,Name,Author,Rating,Reviews,Price,Year,Genre
534,Where the Crawdads Sing,Delia Owens,4.8,87841,15,2019,Fiction
382,The Girl on the Train,Paula Hawkins,4.1,79446,18,2015,Fiction
383,The Girl on the Train,Paula Hawkins,4.1,79446,7,2016,Fiction
32,Becoming,Michelle Obama,4.8,61133,11,2018,Non Fiction
33,Becoming,Michelle Obama,4.8,61133,11,2019,Non Fiction


In [179]:
electronics.head()

,date,number,name,rating,n_reviews,price
0,2021-02-01,1,Fire TV Stick 4K streaming device with Alexa V...,4.7,"512,797",$39.99
1,2021-02-01,2,"Amazon Smart Plug, works with Alexa – A Certif...",4.7,"328,652",$24.99
2,2021-02-01,3,Echo Show 5 -- Smart display with Alexa – stay...,4.6,"298,030",$44.99
3,2021-02-01,4,"All-new Echo Dot (4th Gen, 2020 release) | Sma...",4.7,"138,241",$44.99
4,2021-02-01,5,Roku Streaming Stick+ | HD/4K/HDR Streaming De...,4.7,"62,702",$39.00


### After adding a new dataset containing top 2021 electronics it is evident the date isnt necessary as the book dataset only deals in year values. 

In [180]:
electronics['date'] = pd.DatetimeIndex(electronics['date']).year
electronics = electronics.rename(columns = {'date':'Year'})
electronics.head()

,Year,number,name,rating,n_reviews,price
0,2021,1,Fire TV Stick 4K streaming device with Alexa V...,4.7,"512,797",$39.99
1,2021,2,"Amazon Smart Plug, works with Alexa – A Certif...",4.7,"328,652",$24.99
2,2021,3,Echo Show 5 -- Smart display with Alexa – stay...,4.6,"298,030",$44.99
3,2021,4,"All-new Echo Dot (4th Gen, 2020 release) | Sma...",4.7,"138,241",$44.99
4,2021,5,Roku Streaming Stick+ | HD/4K/HDR Streaming De...,4.7,"62,702",$39.00


### Do we have any null/NaN values in either dataframe?

In [181]:
electronics[electronics.isnull().any(axis=1)] #displaying if any rows have NaN values

,Year,number,name,rating,n_reviews,price


In [182]:
books[books.isnull().any(axis=1)] #displaying if any rows have NaN values

,Name,Author,User Rating,Reviews,Price,Year,Genre


### Since there's no null/NaN values in either data sets I will simply drop any item(s) which have 'rating' values that are not valid/with error

In [183]:
print(electronics.shape,electronics['rating'].dtypes,
books.shape, books['User Rating'].dtypes)

(18100, 6) object (550, 7) float64


In [184]:
#because the rating column is an object it is assumed theres an error in row, 
# so I will drop if character count is over 3 
electronics.drop((electronics[(electronics['rating'].map(lambda calc: len(calc)) > 3)]).index, inplace = True)
electronics['rating'] = electronics['rating'].astype(float, errors = 'raise')
electronics.shape

(17959, 6)

In [185]:
#user rating is a float as expected, so any non float, or under 2.0 wil be dropped (as 2.0 is the lowest rating we will accept)
books = books[(books['User Rating'] > 2.0) | (books['User Rating'] != float) ]
books.shape

(550, 7)

In [186]:
bookReviews =books['Reviews'].mean()
electronics['n_reviews'] = electronics['n_reviews'].str.replace(',','')
#electronics['n_reviews'] = electronics['n_reviews'].astype(float, errors = 'raise')
electronics['n_reviews']= pd.to_numeric(electronics['n_reviews'],errors = 'coerce')
elecReviews = electronics['n_reviews'].mean()
print(bookReviews, elecReviews)
print(electronics.shape,books.shape)

11953.281818181818 77220.90511721143
(17959, 6) (550, 7)


### Only want the values to be included that are above the mean value

In [187]:
books = books[(books['Reviews'] > 11953)]
electronics = electronics[(electronics['n_reviews'] > 77220)]
print(electronics.shape,books.shape)

(3957, 6) (183, 7)


In [188]:
books = books.sort_values('Reviews', ascending=False)
books.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
534,Where the Crawdads Sing,Delia Owens,4.8,87841,15,2019,Fiction
382,The Girl on the Train,Paula Hawkins,4.1,79446,18,2015,Fiction
383,The Girl on the Train,Paula Hawkins,4.1,79446,7,2016,Fiction
32,Becoming,Michelle Obama,4.8,61133,11,2018,Non Fiction
33,Becoming,Michelle Obama,4.8,61133,11,2019,Non Fiction


In [189]:
electronics = electronics.sort_values('n_reviews', ascending=False)
electronics = electronics.rename(columns = {'n_reviews':'Reviews'})
electronics.head()

,Year,number,name,rating,Reviews,price
13376,2021,77,Echo Dot (3rd Gen) - Smart speaker with Alexa ...,4.7,1154421,$34.99
13476,2021,77,Echo Dot (3rd Gen) - Smart speaker with Alexa ...,4.7,1154421,$34.99
13403,2021,4,Echo Dot (3rd Gen) - Smart speaker with Alexa ...,4.7,1154421,$24.99
13305,2021,6,Echo Dot (3rd Gen) - Smart speaker with Alexa ...,4.7,1151460,$24.99
13205,2021,6,Echo Dot (3rd Gen) - Smart speaker with Alexa ...,4.7,1150403,$24.99


In [190]:
electronics = electronics.drop_duplicates(subset=['name', 'rating', 'Reviews'], keep='last')
books = books.drop_duplicates(subset=['Name', 'Author', 'Reviews', 'Year'], keep='last')

### Based on these results versus the books charts I used during milestone 1 I am not doing to drop duplicates if they are different year as the year can indicate the book was re released and shows progress over time <br>
### Renaming column names to match amoung both tables

In [192]:
electronics = electronics.rename(columns = {'number':'Rank'})
electronics = electronics.rename(columns = {'name':'Name'})
electronics = electronics.rename(columns = {'price':'Price'})
electronics = electronics.rename(columns = {'rating':'Rating'})
books = books.rename(columns = {'User Rating':'Rating'})

In [193]:
books.head()

,Name,Author,Rating,Reviews,Price,Year,Genre
534,Where the Crawdads Sing,Delia Owens,4.8,87841,15,2019,Fiction
382,The Girl on the Train,Paula Hawkins,4.1,79446,18,2015,Fiction
383,The Girl on the Train,Paula Hawkins,4.1,79446,7,2016,Fiction
32,Becoming,Michelle Obama,4.8,61133,11,2018,Non Fiction
33,Becoming,Michelle Obama,4.8,61133,11,2019,Non Fiction


In [194]:
electronics.head()

,Year,Rank,Name,Rating,Reviews,Price
13476,2021,77,Echo Dot (3rd Gen) - Smart speaker with Alexa ...,4.7,1154421,$34.99
13403,2021,4,Echo Dot (3rd Gen) - Smart speaker with Alexa ...,4.7,1154421,$24.99
13305,2021,6,Echo Dot (3rd Gen) - Smart speaker with Alexa ...,4.7,1151460,$24.99
13205,2021,6,Echo Dot (3rd Gen) - Smart speaker with Alexa ...,4.7,1150403,$24.99
13274,2021,75,Echo Dot (3rd Gen) - Smart speaker with Alexa ...,4.7,1150403,$24.99
